# Lecture 8: Home storage systems & operational strategies

This notebook explores two different operational strategies to improve a household's PV self-consumption with home battery storage systems: 
* The *greedy* strategy, which charges and discharges the storage whenever available and
* the *feed-in damp*, which aims to limit the power feed during peak PV generation.

A techno-economic analysis is performed with the help of [simses](https://gitlab.lrz.de/open-ees-ses/simses) a simulation framework for stationary energy storage systems.

In [ ]:
import os
import pandas as pd

from simses.main import SimSES
from configparser import ConfigParser

In [ ]:
pd.options.plotting.backend = "plotly"
# template = "plotly_dark"
template = "plotly_white"

# Greedy strategy

### Simulation parameters

In [ ]:
power    = 3000 # W
capacity = 13500 # Wh
loop_years = 10 # a - number of years to simulate via repetetive one-year profile execution

electricity_consumption = 5e6 # Yearly energy consumption in Wh
pv_installed_capacity = 7e3   # PV peak-power in W

sim_params = f"""
[GENERAL]
START = 2014-01-01 00:00:00
END = 2014-12-31 23:59:59
TIME_STEP = 3600
LOOP = {loop_years}

[ENERGY_MANAGEMENT]
STRATEGY = ResidentialPvGreedy

[BATTERY]
START_SOC = 0.5
MIN_SOC = 0.1
MAX_SOC = 0.85

[STORAGE_SYSTEM]
; Configuration of the AC storage system:
; Format: AC-system name, max AC power in W, DC voltage level in V, ACDC converter name, housing name, HVAC name
STORAGE_SYSTEM_AC =
    system_1,{power},333,notton,no_housing,no_hvac

; Configuration of the AC/DC converter:
; Format: ACDC converter name, converter type, optional: number of converters
ACDC_CONVERTER =
    notton,NottonAcDcConverter

; Configuration of the DC storage system. Every AC system must have at least 1 DC system
; Format: AC-system name, DCDC converter name, storage technology name
STORAGE_SYSTEM_DC =
   system_1,no_loss,nmc

; Configuration of the DCDC converter
; Format: DCDC converter name, converter type, [efficiency]
DCDC_CONVERTER =
    no_loss,NoLossDcDcConverter

; Configuration of the storage technology.
; Format: storage technology name, energy in Wh, technology type, [technology specific parameters]
STORAGE_TECHNOLOGY =
    nmc,{capacity},lithium_ion,SanyoNMC

[PROFILE]
POWER_PROFILE_DIR = {os.path.abspath("../data")}
LOAD_PROFILE = simses_load_profile
GENERATION_PROFILE = simses_pv_profile

LOAD_SCALING_FACTOR = {electricity_consumption}
GENERATION_SCALING_FACTOR = {pv_installed_capacity}
"""
sim_config = ConfigParser()
sim_config.read_string(sim_params)

In [ ]:
investment_costs = 6875  # €
electricity_price = 0.32 # €/kWh
pv_feedin_tariff = 0.065 # €/kWh
discount_rate = 0.02

analysis_params = f"""
[ECONOMIC_ANALYSIS]
;Costs in Euro
INVESTMENT_COSTS = {investment_costs}
USE_SPECIFIC_COSTS = False

ELECTRICITY_PRICE = {electricity_price}
PV_FEED_IN_TARIFF = {pv_feedin_tariff}

DISCOUNT_RATE = {discount_rate}
"""
analysis_config = ConfigParser()
analysis_config.read_string(analysis_params)

In [ ]:
path = os.path.abspath("..")
result_path = os.path.join(path, "simses_results").replace("\\", "/") + "/"

### Run simulation

In [ ]:
simses = SimSES(path=result_path, name="greedy", simulation_config=sim_config, analysis_config=analysis_config)

In [ ]:
simses.run()

### Load results

In [ ]:
# results path
results_greedy = os.path.join(result_path, "greedy")
latest = os.listdir(results_greedy)[-1]
results_greedy = os.path.join(results_greedy, latest).replace("\\", "/")

In [ ]:
df_greedy_sys = pd.read_csv(results_greedy + "/SystemState.csv.gz").drop_duplicates("Time in s")
df_greedy_ems = pd.read_csv(results_greedy + "/EnergyManagementState.csv.gz")
df_greedy_lis = pd.read_csv(results_greedy + "/LithiumIonState.csv.gz")

In [ ]:
time = df_greedy_ems["Time in s"]
load = df_greedy_ems["Load in W"].values
pv_gen = df_greedy_ems["PV Generation in W"].values
residual_damp = load - pv_gen

power_greedy = df_greedy_sys["AC_P_delivered in W"].values
soc_greedy   = df_greedy_sys["SOC in p.u."].values

df_greedy = pd.DataFrame(
    data={
        "Residual in W": residual_damp,
        "Storage power in W": -power_greedy,
        "Storage SOC in p.u.": soc_greedy,
        "Grid power in W": residual_damp + power_greedy
    },
    index = pd.to_datetime(time, unit="s")
)

## Feed-in damp

### Parametrize and run simulation

In [ ]:
sim_config.set("ENERGY_MANAGEMENT", "STRATEGY", "ResidentialPvFeedInDamp")
simses = SimSES(path=result_path, name="feedindamp", simulation_config=sim_config, analysis_config=analysis_config)

In [ ]:
simses.run()

### Load results

In [ ]:
results_damp = os.path.join(result_path, "feedindamp")
latest = os.listdir(results_damp)[-1]
results_damp = os.path.join(results_damp, latest).replace("\\", "/")

In [ ]:
df_damp_sys = pd.read_csv(results_damp + "/SystemState.csv.gz").drop_duplicates("Time in s")
df_damp_ems = pd.read_csv(results_damp + "/EnergyManagementState.csv.gz")
df_damp_lis = pd.read_csv(results_damp + "/LithiumIonState.csv.gz")

In [ ]:
time = df_damp_ems["Time in s"]
load = df_damp_ems["Load in W"].values
pv_gen = df_damp_ems["PV Generation in W"].values
residual_damp = load - pv_gen

power_damp = df_damp_sys["AC_P_delivered in W"].values
soc_damp   = df_damp_sys["SOC in p.u."].values

df_damp = pd.DataFrame(
    data={
        "Residual in W": residual_damp,
        "Storage power in W": -power_damp,
        "Storage SOC in p.u.": soc_damp,
        "Grid power in W": residual_damp + power_damp
    },
    index = pd.to_datetime(time, unit="s")
)

## Analysis

In [ ]:
# join results in a single dataframe
df = df_greedy.join(
    df_damp[["Storage power in W", "Storage SOC in p.u.", "Grid power in W"]], 
    lsuffix=" (Greedy)", 
    rsuffix=" (Feed-in Damp)"
)

### Storage power

In [ ]:
df[["Residual in W", "Storage power in W (Greedy)", "Storage power in W (Feed-in Damp)"]].plot(template=template, labels={"value": "Storage power in W"})

In [ ]:
df[["Storage power in W (Greedy)", "Storage power in W (Feed-in Damp)"]].plot.hist(template=template, log_y=True, labels={"value": "Storage power in W"}).update_layout(barmode='overlay').update_traces(opacity=0.75)

### State of Charge (SOC)

In [ ]:
df[["Storage SOC in p.u. (Greedy)", "Storage SOC in p.u. (Feed-in Damp)"]].plot(template=template, labels={"value": "SOC in p.u."})

In [ ]:
df[["Storage SOC in p.u. (Greedy)", "Storage SOC in p.u. (Feed-in Damp)"]].plot.hist(template=template, log_y=True, labels={"value": "SOC in p.u."}).update_layout(barmode='overlay').update_traces(opacity=0.75)


### Grid power

In [ ]:
df[["Grid power in W (Greedy)", "Grid power in W (Feed-in Damp)"]].plot(template=template, labels={"value": "Grid power in W"})

In [ ]:
df[["Grid power in W (Greedy)", "Grid power in W (Feed-in Damp)"]].plot.hist(template=template, log_y=True, labels={"value": "Grid power in W"}).update_layout(barmode='overlay').update_traces(opacity=0.75)